In [29]:
import os
import numpy as np
import gensim

In [30]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname) as f:
        for i, line in enumerate(f):
            tokens = line.split()
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus('ogbg-molhiv.txt'))

In [31]:
len(train_corpus)

41127

In [32]:
model = gensim.models.doc2vec.Doc2Vec(
    vector_size=1000,
    window=5
    )

In [33]:
model.build_vocab(train_corpus)

In [34]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [35]:
X = [model.dv[node] for node in range(len(model.dv))]

In [36]:
from ogb.graphproppred import GraphPropPredDataset

###

d_name = 'ogbg-molhiv'
dataset = GraphPropPredDataset(name=d_name)
split_idx = dataset.get_idx_split()

y = [data[1] for data in dataset]

/home/cstaib/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
train_idx = split_idx['train']
test_idx = split_idx['test']

train_X = np.array([X[i] for i in range(len(X)) if i in train_idx])
test_X = np.array([X[i] for i in range(len(X)) if i in test_idx])

train_y = np.array([y[i] for i in range(len(y)) if i in train_idx]).ravel()
test_y = np.array([y[i] for i in range(len(y)) if i in test_idx]).ravel()

In [38]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_X, train_y)

RandomForestClassifier()

In [39]:
train_y.shape

(32901,)

In [40]:
y_pred = clf.predict(test_X)

In [41]:
np.bincount(y_pred)

array([4110,    3])

In [42]:
np.bincount(test_y)

array([3983,  130])

In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, clf.predict(test_X))

0.5035950868112555